# Shared LSTM Model - SNLI Dataset

The following notebook contains the implementation of the shared LSTM model for the SNLI dataset

In [1]:
# Import the necessary libraries

import re
from numpy import array
from numpy import asarray
from numpy import zeros

import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.regularizers import L2
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Dropout, Input, LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping, ModelCheckpoint

import nltk
nltk.download('punkt')

from gensim import models

[nltk_data] Downloading package punkt to /home/yss/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Helper function that cleans the input data and enumerates the labels

def extract(s):
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()

labels = {'entailment': 0, 'contradiction': 1, 'neutral': 2}

In [3]:
# Function that reads data and parses data from file

def readFileData(filePath):
    with open(filePath, 'r') as f:
        inputRows = [row.split('\t') for row in f.readlines()[1:]]

    inputPremises = [extract(row[1]) for row in inputRows if row[0] in labels]
    inputHypotheses = [extract(row[2]) for row in inputRows if row[0] in labels]
    inputLabels = [labels[row[0]] for row in inputRows if row[0] in labels]
    f.close()

    return [inputPremises, inputHypotheses, inputLabels]

In [4]:
# Reading train and test data

datasetPath = "../../Datasets/SNLI/"
trainData = readFileData(f'{datasetPath}snli_1.0_train.txt')
testData = readFileData(f'{datasetPath}snli_1.0_test.txt')
validationData = readFileData(f'{datasetPath}snli_1.0_dev.txt')

In [5]:
# Hyperparameters

maxLen = 50
epochs = 1000
batchSize = 128
gloveDimension = 300
hiddenDimension = 100
regularization = 4e-6

In [6]:
# Tokenizer to generate the vocabulary of the system

tokenizer = Tokenizer()
tokenizer.fit_on_texts(trainData[0] + trainData[1])
vocabSize = len(tokenizer.word_index)+1

In [7]:
# Convert the train data to sequences as per the vocabulary
trainData[0] = tokenizer.texts_to_sequences(trainData[0])
trainData[1] = tokenizer.texts_to_sequences(trainData[1])

# Pad or trim all generated sequences to the same max sentence length
trainData[0] = pad_sequences(trainData[0], maxLen, padding='post')
trainData[1] = pad_sequences(trainData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
trainData[2] = tf.keras.utils.to_categorical(trainData[2], num_classes=3)

In [8]:
# Convert the test data to sequences as per the vocabulary
testData[0] = tokenizer.texts_to_sequences(testData[0])
testData[1] = tokenizer.texts_to_sequences(testData[1])

# Pad or trim all generated sequences to the same max sentence length
testData[0] = pad_sequences(testData[0], maxLen, padding='post')
testData[1] = pad_sequences(testData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
testData[2] = tf.keras.utils.to_categorical(testData[2], num_classes=3)

In [9]:
# Convert the validation data to sequences as per the vocabulary
validationData[0] = tokenizer.texts_to_sequences(validationData[0])
validationData[1] = tokenizer.texts_to_sequences(validationData[1])

# Pad or trim all generated sequences to the same max sentence length
validationData[0] = pad_sequences(validationData[0], maxLen, padding='post')
validationData[1] = pad_sequences(validationData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
validationData[2] = tf.keras.utils.to_categorical(validationData[2], num_classes=3)

In [10]:
# Generate the embeddings dictionary from Google News word2vec

word2vecpath = "../../Datasets/Word2Vec/"
embeddingsDict = models.KeyedVectors.load_word2vec_format(f'{word2vecpath}GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
# Alternative GloVe embeddings

'''
embeddingsDict = dict()
glove = open(r'/content/drive/MyDrive/glove.840B.300d.txt', encoding='utf8')

for line in glove:
    records = line.split()
    word = ''.join(records[:-300])
    vectorDimensions = asarray(records[-300:], dtype='float32')
    embeddingsDict[word] = vectorDimensions

glove.close()
'''

In [11]:
# Iterate through the embeddings and store only those that are present in our vocabulary
'''
embeddingsMat = zeros((vocabSize, gloveDimension))
for word, index in tokenizer.word_index.items():
    vec = embeddingsDict.get(word)
    if vec is not None:
        embeddingsMat[index] = vec
'''

embeddingsMat = zeros((vocabSize, gloveDimension))
for word, index in tokenizer.word_index.items():
    if index % 2500 == 0:
        print(index)
    if word in embeddingsDict.index_to_key:
        embeddingsMat[index] = embeddingsDict[word]

2500
5000
7500
10000
12500
15000
17500
20000
22500
25000
27500
30000
32500


In [ ]:
# Define the embedding layer for our baseline RNN model
embed = Embedding(vocabSize, gloveDimension, weights=[embeddingsMat], input_length=maxLen, trainable=False)

# As Premise and Hypothesis are distinct and are to be inputted separately, define two inputs and embed
premise = Input(shape=(maxLen,), dtype='int32')
hypothesis = Input(shape=(maxLen,), dtype='int32')

premInput = embed(premise)
hypoInput = embed(hypothesis)

convert = Dense(hiddenDimension, activation='tanh', input_shape=(gloveDimension,))

premInput = convert(premInput)
hypoInput = convert(hypoInput)

In [13]:
# Once the sentence embeddings have been generated, generate a matrix of dimensions maxLen X gloveDimension
# On adding maxLen, we get a single embedding vector of length gloveDimension

#rnn = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=hiddenDimension)
#rnn = LSTM(hiddenDimension, dropout=0.2, recurrent_dropout=0.2)

rnnH = LSTM(hiddenDimension, dropout=0.2, return_state=True)
rnnP = LSTM(hiddenDimension, dropout=0.2)

In [14]:
# Apply batch normalization to the two input embeddings separately

premInput, sH, sC = rnnH(premInput)
hypoInput = rnnP(hypoInput, initial_state=[sH, sC])
premInput = tf.keras.layers.BatchNormalization()(premInput)
result = tf.keras.layers.BatchNormalization()(hypoInput)

In [15]:
# Joint is a concatenated embeddings layer, generated from the premise and hypothesis inputs
# Dilution of probability 0.2, to assist in regularization
#joint = keras.layers.concatenate([premInput, hypoInput])
result = Dropout(0.2)(result)
'''
for i in range(3):
    result = Dense(2*hiddenDimension, activation='tanh', kernel_regularizer=L2(regularization))(result)
    result = Dropout(0.2)(result)
    result = tf.keras.layers.BatchNormalization()(result)
'''

# 3 layers of the TanH activation function, along with L2 regularization.
# The final decision is based on the Softmax function

result = Dense(hiddenDimension, activation='tanh', kernel_regularizer=L2(regularization))(result)
pred = Dense(3, activation='softmax')(result)

In [23]:
# Defining the final models input and output format, as well as compilation parameters

model = Model(inputs=[premise, hypothesis], outputs=pred)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [1]:
# Fitting the model using the train data

callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)
mcCallback = ModelCheckpoint('./2LSTM_snli.h5', monitor='val_loss')
model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=batchSize, epochs=epochs, callbacks=[callback, mcCallback], validation_data=[[array(validationData[0]), array(validationData[1])], array(validationData[2])])


Epoch 1/1000
4292/4292 [==============================] - 197s 46ms/step - loss: 0.4794 - accuracy: 0.8116 - val_loss: 0.5099 - val_accuracy: 0.8056
Epoch 2/1000
4292/4292 [==============================] - 254s 59ms/step - loss: 0.4804 - accuracy: 0.8111 - val_loss: 0.5064 - val_accuracy: 0.8079
Epoch 3/1000
4292/4292 [==============================] - 253s 59ms/step - loss: 0.4800 - accuracy: 0.8117 - val_loss: 0.5021 - val_accuracy: 0.8076
Epoch 4/1000
4292/4292 [==============================] - 257s 60ms/step - loss: 0.4791 - accuracy: 0.8114 - val_loss: 0.5008 - val_accuracy: 0.8063
Epoch 5/1000
4292/4292 [==============================] - 248s 58ms/step - loss: 0.4801 - accuracy: 0.8114 - val_loss: 0.5033 - val_accuracy: 0.8034
Epoch 6/1000
4292/4292 [==============================] - 226s 53ms/step - loss: 0.4807 - accuracy: 0.8107 - val_loss: 0.5013 - val_accuracy: 0.8042
Epoch 7/1000
4292/4292 [==============================] - 219s 51ms/step - loss: 0.4806 - accuracy: 0.811

In [ ]:
model.save('../../Models/2LSTM_snli.h5')

In [38]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)

39/39 [==============================] - 1s 30ms/step - loss: 0.5140 - accuracy: 0.8005
Loss =  0.514041006565094
Acc =  0.8004885911941528


In [39]:
ccc = keras.models.load_model('../../Models/2LSTM_snli.h5')
l, a = ccc.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print(a)

39/39 [==============================] - 2s 28ms/step - loss: 0.5106 - accuracy: 0.8013
0.8013029098510742


We can thus observe the following results for the shared LSTM model, for the SNLI dataset:

- Training Accuracy: 81.15%
- Validation Accuracy: 80.20%
- Test Accuracy: 80.13%

Here, we see an increase in performance using the shared LSTM model, as compared to the other datasets. As SNLI is a large dataset, we can infer this improvement of performance over SciTail and SICK.